# A notebook to demonstrate usage of the Jupyter Widget for the Cell Distance Explorer

In this notebook, we download a sample dataset with 3D cell positions from the [Cell Distance Explorer (CDE)](https://apps.humanatlas.io/cde/) and then visualize if with the `node-dist-vis` and `CDE` Jupyter widgets.

# Install and import libraries

In [1]:
# Import native packages
import os
from pprint import pprint

In [2]:
#Install and import external packages
%pip install requests pandas ipywidgets hra_jupyter_widgets

import requests
import pandas as pd
import ipywidgets as widgets

# Import hra-jupyter-widgets. For documentation, please see https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb
from hra_jupyter_widgets import (
    # BodyUi,
    CdeVisualization, # in this example, we will use this one...
    # Eui,
    # EuiOrganInformation,
    # FtuExplorer,
    # FtuExplorerSmall,
    # MedicalIllustration,
    # ModelViewer,
    NodeDistVis, # ...and this one, but all of them are usable for different purposes!
    # Rui,
)


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Global variables

In [3]:
output_file = "sample_data.csv"
output_folder = "data"
path = os.path.join(output_folder, output_file)

# Download data

In [4]:
def download_public_sheet_as_csv(file_id, gid, path):
    
    # ensure output folder exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    url = (
        f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gid}"
    )
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, "wb") as f:
            f.write(response.content)
        print(f"Success: saved to {path}")
    else:
        print(f"Error {response.status_code}: could not download sheet")


# Your sheet’s IDs
file_id = "1EUf7CUZb0NprgxBeX3nS3GXyUM89edIRH6NSeUsR3nY"
gid = "47698780"

download_public_sheet_as_csv(file_id, gid,path)

Success: saved to data/sample_data.csv


## Read data as DataFrame

In [5]:
# Read the CSV file and convert it to a df
df = pd.read_csv(path)
df

,x,y,z,Cell Type
0,188.500,342.500,10,T-Killer
1,136.600,161.400,20,T-Killer
2,437.500,363.500,20,T-Killer
3,203.500,512.000,20,T-Killer
4,155.579,204.895,0,T-Helper
...,...,...,...,...
2863,155.500,133.500,20,DDB2
2864,102.500,251.500,20,DDB2
2865,133.905,378.381,20,DDB2
2866,439.846,378.231,20,DDB2


## Visualize wit CDE

In [6]:
# Next, let's define a function that turns a DataFrame into a node list that can then be passed into the CdeVisualization or NodeDistVis widget
def make_node_list(df: pd.DataFrame, is_3d: bool = False):
    """Turn a DataFrame into a list of dicts for passing them into a HRA widget

    Args:
        df (pd.DataFrame): A DataFrame with cells
    """

    # If the df does not have a z-axis column, let's add one and set all cells to 0
    if not is_3d:
        df.loc[:, ("z")] = 0
    
    node_list = [
        {"x": row["x"], "y": row["y"], "z": row["z"], "Cell Type": row["Cell Type"]}
        for index, row in df.iterrows()
    ]

    return node_list

In [7]:
# Prepare df_cells for visualization with NodeDistVis widget
node_list = make_node_list(df, True)

# Let's inspect the first 5 rows
pprint(node_list[:5])

[{'Cell Type': 'T-Killer', 'x': 188.5, 'y': 342.5, 'z': 10},
 {'Cell Type': 'T-Killer', 'x': 136.6, 'y': 161.4, 'z': 20},
 {'Cell Type': 'T-Killer', 'x': 437.5, 'y': 363.5, 'z': 20},
 {'Cell Type': 'T-Killer', 'x': 203.5, 'y': 512.0, 'z': 20},
 {'Cell Type': 'T-Helper', 'x': 155.579, 'y': 204.895, 'z': 0}]


In [8]:
# Finally, let's instantiate the NodeDistVis class with some parameters. We pass in the node_list, indicate Endothelial cells as targets for the edges.
# As we are not supplying an edge list, we need to provide a max_edge_distance, which is set to 1000 (generiously)
node_dist_vis = NodeDistVis(
    nodes=node_list,
    node_target_key="Cell Type",
    node_target_value="Endothelial",
    max_edge_distance=1000,
)

# Display our new widget
display(node_dist_vis)

NodeDistVis(color_map=None, color_map_data=None, color_map_key=None, edges=None, max_edge_distance=1000, node_…

In [9]:
# Finally, let's instantiate the CDEVisualization class with our node_list as parameter.
cde = CdeVisualization(nodes=node_list)

# Display our new widget
display(cde)

CdeVisualization(age=None, color_map=None, color_map_key=None, color_map_value_key=None, creation_timestamp=No…